# Import Required Libraries

Importing the necessary libraries:


In [1]:
!pip install transformers

In [3]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from transformers import DistilBertTokenizer

from transformers import DistilBertForSequenceClassification

# Load the Dataset

Load the dataset into a pandas DataFrame


In [4]:
# Load train data
train_data = pd.read_csv('https://raw.githubusercontent.com/salarMokhtariL/Facke-News-Detection/main/Dataset/train.csv')

# Load test data
test_data = pd.read_csv('https://raw.githubusercontent.com/salarMokhtariL/Facke-News-Detection/main/Dataset/test.csv')

In [5]:
train_data.dropna(inplace=True)


# Prepare the Data
Prepare the data for the PyTorch model. First, let's define a custom dataset class

In [6]:
''' This class takes in the data, tokenizes it using the DistilBertTokenizer from the transformers library,
 and returns the input IDs, attention masks, and labels.'''


class FakeNewsDataset(Dataset):
    def __init__(self, data, max_len=128):
        self.data = data
        self.max_len = max_len
        self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = self.data.iloc[index]['text']
        label = self.data.iloc[index]['label']
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_token_type_ids=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return inputs['input_ids'].squeeze(0), inputs['attention_mask'].squeeze(0), torch.tensor(label, dtype=torch.long)

In [7]:
# split the data into training and validation sets

train_data, val_data = train_test_split(train_data, test_size=0.2,
                                        random_state=42)

In [8]:
# Create PyTorch data loaders for the training, validation, and test sets:

train_dataset = FakeNewsDataset(train_data)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = FakeNewsDataset(val_data)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

test_dataset = FakeNewsDataset(test_data)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the Model
define the PyTorch model. We'll use the `DistilBertForSequenceClassification` model from the `transformers` library:

In [9]:
class FakeNewsClassifier(nn.Module):
    def __init__(self, num_labels=2):
        super(FakeNewsClassifier, self).__init__()
        self.bert = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs[0]

# Train the Model

With the data and model prepared, we can now train the model using PyTorch. We'll define a function to train the model for one epoch

In [10]:
def train_epoch(model, optimizer, criterion, train_loader):
    model.train()
    train_loss = 0
    train_acc = 0

    for input_ids, attention_mask, labels in tqdm(train_loader, desc='Training'):
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids.to(device), attention_mask=attention_mask.to(device))
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += (outputs.argmax(1) == labels.to(device)).sum().item()

    train_loss /= len(train_loader)
    train_acc /= len(train_loader.dataset)

    return train_loss, train_acc

This function takes in the model, optimizer, loss function, and data loader, and performs a forward pass through the model, calculates the loss, and performs backpropagation and gradient descent to update the model parameters

We'll also define a function to evaluate the model on the validation set:

In [11]:
def eval_epoch(model, criterion, val_loader):
    model.eval()
    val_loss = 0
    val_acc = 0

    with torch.no_grad():
        for input_ids, attention_mask, labels in tqdm(val_loader, desc='Validation'):
            outputs = model(input_ids=input_ids.to(device), attention_mask=attention_mask.to(device))
            loss = criterion(outputs, labels.to(device))

            val_loss += loss.item()
            val_acc += (outputs.argmax(1) == labels.to(device)).sum().item()

        val_loss /= len(val_loader)
        val_acc /= len(val_loader.dataset)

    return val_loss, val_acc

This function takes in the model, loss function, and data loader, and performs a forward pass through the model to calculate the loss and accuracy on the validation set.

Now we can define the main training loop:

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = FakeNewsClassifier().to(device)
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

best_val_acc = 0

for epoch in range(5):
    train_loss, train_acc = train_epoch(model, optimizer, criterion, train_loader)
    val_loss, val_acc = eval_epoch(model, criterion, val_loader)

    print(f'Epoch {epoch + 1}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}')

    if val_acc > best_val_acc:
        torch.save(model.state_dict(), 'best_model.pt')
        best_val_acc = val_acc

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Training:  11%|█         | 49/458 [06:02<50:26,  7.40s/it]


KeyboardInterrupt: 

In [ ]:
# Train and evaluate the model
epochs = 3

for epoch in range(epochs):
    # Train
    model.train()
    total_train_loss = 0
    for step, batch in enumerate(train_loader):
        batch_inputs, batch_masks, batch_labels = batch
        optimizer.zero_grad()
        outputs = model(batch_inputs, attention_mask=batch_masks)
        loss = loss_fn(outputs, batch_labels)
        total_train_loss += loss.item()
        loss.backward()
        optimizer.step()
        if step % 100 == 0:
            print(f"Epoch {epoch+1} / {epochs} - Batch {step} / {len(train_loader)} - Loss: {loss.item()}")

    # Evaluate
    model.eval()
    total_validation_loss = 0
    predictions, true_labels = [], []
    for batch in val_loader:
        batch_inputs, batch_masks, batch_labels = batch
        with torch.no_grad():
            outputs = model(batch_inputs, attention_mask=batch_masks)
            loss = loss_fn(outputs, batch_labels)
            total_validation_loss += loss.item()
            predictions += list(torch.argmax(outputs, dim=1).cpu().numpy())
            true_labels += list(batch_labels.cpu().numpy())

    # Print training and validation loss
    average_train_loss = total_train_loss / len(train_loader)
    average_validation_loss = total_validation_loss / len(val_loader)
    print(f"Epoch {epoch+1} / {epochs} - Average training loss: {average_train_loss}")
    print(f"Epoch {epoch+1} / {epochs} - Average validation loss: {average_validation_loss}")

    # Print classification report
    print(classification_report(true_labels, predictions))

    # Save model
    torch.save(model.state_dict(), f"distilbert-fake-news-{epoch+1}.pth")